In [10]:
#import and config enviroment
import pandas as pd
import sys
import re

sys.path.insert(0, "../lib")

import os
import nltk
from nltk.tree import *

# Lexical Parser
from nltk.parse import CoreNLPParser

parser = CoreNLPParser(url='http://localhost:9000')

# Neural Dependency Parser
from nltk.parse.corenlp import CoreNLPDependencyParser
dep_parser = CoreNLPDependencyParser(url='http://localhost:9000')

os.environ["NLTK_DATA"] = "../"
nltk.data.path.append("../nltk_data")

In [11]:
def getWantedTags():
    return ["JJ"]

In [12]:
def getCleaningPatterns():
    return [
                [r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', "", True],
                [r'[\w\.-]+@[\w\.-]+', "", True],
                ["\"",   "", False],
                [".", " . ", False],
                [",", " , ", False],
                [":", " : ", False],
                [";", " ; ", False],
                ["?", " ? ", False],
                ["!", " ! ", False],
                [" '",  " ", False],
                ["' ",  " ", False]
            ]

In [13]:
def cleanText(text):
    for i in getCleaningPatterns():
        if i[2]:
            text = re.sub(i[0], i[1], text, flags = re.MULTILINE)
        else:
            text = text.replace(i[0], i[1])
            
    return text

In [14]:
def tokenizeComment(text):
    tokens = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(tokens)
    
    return tagged

In [15]:
def getWantedTagsFromAllTags(all_list, wanted_list):
    clean = []
    
    for i in all_list:
        for j in wanted_list:
            if i[1] == j:
                clean.append(i[0])
    
    return clean

In [29]:
#import the comments
df_comment = pd.read_csv('../data/dataset.csv',sep = "\t", header = None, encoding = "utf8")
df_comment = df_comment.rename(index = int, columns = { 0 : "comment"})

#import the scors
df_score   = pd.read_csv('../data/labels.csv',sep = "\t", header = None, encoding = "utf8")
df_score   = df_score.rename(index = int, columns = { 0 : "result"})

#merge comments and scores into one dataframe
df = pd.concat([df_comment, df_score], axis = 1)

In [30]:
line = 4000

#get the wanted comment
sentence = (df["comment"][line])

#clean the unwanted parts
sentence = cleanText(sentence)

#transform the comment to a list of tags
all_tags = tokenizeComment(sentence)

#get the wanted tags
clean_tags = getWantedTagsFromAllTags(all_tags, getWantedTags())
            
#print("score : ", df["score"][line])
#display(clean_tags)


In [31]:
# Parse tokenized text.
#list(parser.parse('What is the airspeed of an unladen swallow ?'.split()))
#dp = Tree('dp', test)
#display(dp)


# Parse raw string.
#list(parser.raw_parse('What is the airspeed of an unladen swallow ?'))


# Neural Dependency Parser
#parses = list(dep_parser.parse('i am not into good things'.split()))

# Tokenizer
#parser = CoreNLPParser(url='http://localhost:9000')
#list(parser.tokenize('What is the airspeed of an unladen swallow?'))


# POS Tagger
#pos_tagger = CoreNLPParser(url='http://localhost:9000', tagtype='pos')
#list(pos_tagger.tag('What is the airspeed of an unladen swallow ?'.split()))


# NER Tagger
#ner_tagger = CoreNLPParser(url='http://localhost:9000', tagtype='ner')
#list(ner_tagger.tag(('Rami Eid is studying at Stony Brook University in NY'.split())))


#Test

#tree = Tree('dp', list(parser.raw_parse("What is the highest waterfall in the United States ?")))
#ptree = ParentedTree.convert(tree[0])

# display(Tree('dp', list(parser.raw_parse("exemple d'une phrase en anglais"))))
# display(Tree('dp', list(parser.raw_parse("That's not really bad."))))
# display(Tree('dp', list(parser.raw_parse("i am not into good things"))))
# display(Tree('dp', list(parser.raw_parse("not only this is good, but something"))))
# display(Tree('dp', list(parser.raw_parse("i don't like bad boys'"))))

mytree = list(parser.raw_parse("That's not really bad."))
ptree = ParentedTree.convert(mytree[0])
display(mytree)

[Tree('ROOT', [Tree('S', [Tree('NP', [Tree('DT', ['That'])]), Tree('VP', [Tree('VBZ', ["'s"]), Tree('RB', ['not']), Tree('ADJP', [Tree('RB', ['really']), Tree('JJ', ['bad'])])]), Tree('.', ['.'])])])]